# SEN163A - Fundamentals of Data Analytics
# Assignment 3 - Large-scale Internet Data Analysis
### Ir. Jacopo De Stefani - [J.deStefani@tudelft.nl](mailto:J.deStefani@tudelft.nl)
### Joao Pizani Flor, M.Sc. - [J.p.pizaniflor@tudelft.nl](mailto:J.p.pizaniflor@tudelft.nl)

### 25-03-2022
## Group 2
- Emmanuel M Boateng - '5617642'
- Joost Oortwijn - '4593472'
- Philip Busscher - ''4611993''
- Floris Kool - ''4975243''


# Introduction
short description of the assignment

# 1. Dataset description

Short description of the 4 datasets used

## 1.1 Opening the data

In [3]:
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin
import pandas as pd
import time
import pickle

In [10]:

#READING MAIN PAGE
#Get file
source_link = "https://jdestefani.github.io/SEN163A-TabularRazorArchives/"
response = requests.get(source_link)

#Read main file
main_page = BeautifulSoup(response.text)

for year_link in main_page.find_all('a', href=True):
    #READING YEAR PAGE & SAVING PKL FILE FOR EACH YEAR
    
    #Get file name of pkl file
    year_filename = year_link.get('href')
    year_filename = year_filename.removeprefix('./')
    year_filename = year_filename.removesuffix('.html')
    year_filename = "data/" + year_filename + ".pkl"
    
    #Clear lists of data for each year
    Data_Tuple = []
    
    #Get file  
    year_link = urljoin(source_link, year_link.get('href')) 
    response = requests.get(year_link)
    
    #Read year file
    year_page = BeautifulSoup(response.text)
     
    for month_link in year_page.find_all('a', href=True):
        #Get file
        month_link = urljoin(year_link, month_link.get('href'))
        response = requests.get(month_link)
        
        #Read month file
        month_page = BeautifulSoup(response.text)
        i = 0
        
        start  = time.time()
        
        for article_link in month_page.find_all('a', href=True):        
            #Get file
            article_link = urljoin(month_link, article_link.get('href')) 
            response = requests.get(article_link)
        
            #Read article file
            article_page = BeautifulSoup(response.text)
                       
            for div_element in article_page.find_all('div'):
                if div_element.get('class') == ['author']:
                    article_author = div_element.get_text()
                if div_element.get('class') == ['date']:
                    article_date = div_element.get_text()
                if div_element.get('class') == ['time']:
                    article_time = div_element.get_text()

            #Add data
            Data_Tuple.append((article_author, article_date, article_time))
            i += 1 
            if i >= 10:
                break
        
        #Print after each month
        dur = round(time.time() - start,2)
        print('Read: ' + month_link)
        print('Added: ' + str(i) + ' articles')
        print('Duration: ' + str(dur) + ' seconds')
        

    #Save file after each year
    print()
    print("---")
    print("Saving: " + year_filename)
    df = pd.DataFrame(Data_Tuple)
    df.to_pickle(year_filename)
    print("Saved: " + str(len(df[0])) + " articles")
    print("---")
    print()
    
    #break
    

Read: https://jdestefani.github.io/SEN163A-TabularRazorArchives/2012-1.html
Added: 10 articles
Duration: 0.3 seconds
Read: https://jdestefani.github.io/SEN163A-TabularRazorArchives/2012-2.html
Added: 10 articles
Duration: 0.29 seconds
Read: https://jdestefani.github.io/SEN163A-TabularRazorArchives/2012-3.html
Added: 10 articles
Duration: 0.3 seconds
Read: https://jdestefani.github.io/SEN163A-TabularRazorArchives/2012-4.html
Added: 10 articles
Duration: 0.31 seconds
Read: https://jdestefani.github.io/SEN163A-TabularRazorArchives/2012-5.html
Added: 10 articles
Duration: 0.31 seconds
Read: https://jdestefani.github.io/SEN163A-TabularRazorArchives/2012-6.html
Added: 10 articles
Duration: 0.29 seconds
Read: https://jdestefani.github.io/SEN163A-TabularRazorArchives/2012-7.html
Added: 10 articles
Duration: 0.29 seconds
Read: https://jdestefani.github.io/SEN163A-TabularRazorArchives/2012-8.html
Added: 10 articles
Duration: 0.28 seconds
Read: https://jdestefani.github.io/SEN163A-TabularRazorArc

Read: https://jdestefani.github.io/SEN163A-TabularRazorArchives/2017-9.html
Added: 10 articles
Duration: 1.31 seconds
Read: https://jdestefani.github.io/SEN163A-TabularRazorArchives/2017-10.html
Added: 10 articles
Duration: 1.33 seconds
Read: https://jdestefani.github.io/SEN163A-TabularRazorArchives/2017-11.html
Added: 10 articles
Duration: 1.32 seconds
Read: https://jdestefani.github.io/SEN163A-TabularRazorArchives/2017-12.html
Added: 10 articles
Duration: 1.3 seconds

---
Saving: data/2017.pkl
Saved: 120 articles
---

Read: https://jdestefani.github.io/SEN163A-TabularRazorArchives/2018-1.html
Added: 10 articles
Duration: 1.33 seconds
Read: https://jdestefani.github.io/SEN163A-TabularRazorArchives/2018-2.html
Added: 10 articles
Duration: 1.42 seconds
Read: https://jdestefani.github.io/SEN163A-TabularRazorArchives/2018-3.html
Added: 10 articles
Duration: 1.34 seconds
Read: https://jdestefani.github.io/SEN163A-TabularRazorArchives/2018-4.html
Added: 10 articles
Duration: 1.33 seconds
Re

More detailed description of data if needed (Can also be after opening each dataset)

## 1.2 Limitations in data

Evaluate if there are limitations in the provided datasets (AS and probe data set). If you find limitations, describe these and conjecture possible reasons, supported with data.

...

In [9]:
#Code needed to prove limitations

Some list of limitations in text

# 2 Analysis

Short description of what is going to be analyzed

## 2.1 Couple among employees (Question a)

Are there couples among the employees. If so, who? Are they still together?


In [1]:
#Code for question a

Description of results

## 2.2 Children (Question b)
Did any of the employees have a child? If so, who?

In [2]:
#Code for question b

Description of results

## 2.3 Holidays (Question c)
If you would be looking to work for Tabularazor Inc., how many holidays can you expect to get per
year?


In [ ]:
#Code for question c

Description of results

# 3 Conclusion

Add code if needed